In [1]:
# Training notebook for 1Dconv 16x16 Slim model with filtered Dataset3sr and input dig.
# Note that it is currently configured to work with 2 timeslices

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.utils import Sequence
from keras.layers import Conv2D, MaxPooling2D
from qkeras import *

from keras.utils import Sequence
from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping

import os
import random

pi = 3.14159265359

maxval=1e9
minval=1e-9

2025-07-21 21:12:59.186186: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-21 21:12:59.186264: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-21 21:12:59.187459: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-21 21:12:59.194647: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-21 21:13:15.548119: W tensorflow/compiler/tf2

In [2]:
from slim_1D.Slim_SSE import *
from slim_1D.Slim1D_model import *
import utils
from ODG_v2 import OptimizedDataGenerator

2025-07-21 21:14:20.111299: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37110 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB MIG 7g.40gb, pci bus id: 0000:81:00.0, compute capability: 8.0


In [3]:
batch_size = 5000
val_batch_size = 5000
train_file_size = 68
val_file_size = 12

Training

In [4]:
def count_files_in_directory(directory_path):
    try:
        return len([f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))])
    except FileNotFoundError:
        print(f"Directory not found: {directory_path}")
        return 0


In [5]:
dataset_base_dir = "/depot/cms/users/callea/largerWindowPreliminary/dataset_3sr_16x16_50x12P5_parquets/"

data_directory_path = os.path.join(dataset_base_dir, "train_filtered")
data_directory_path_2 = os.path.join(dataset_base_dir, "test_filtered")

tfrecords_dir_train = "/depot/cms/users/callea/largerWindowPreliminary/dataset_3sr_16x16_50x12P5_parquets/train_filtered/"
tfrecords_dir_validation = "/depot/cms/users/callea/largerWindowPreliminary/dataset_3sr_16x16_50x12P5_parquets/test_filtered/"

In [6]:
# Count files in both directories
data_files_count = count_files_in_directory(data_directory_path)

# Print results
print(f"Number of files in data directory: {data_files_count}")


Number of files in data directory: 80


Create a new tfrecord can avoid the problem of not converging. 

In [7]:
validation_generator = OptimizedDataGenerator(
    # Base directory where your parquet files are located
    dataset_base_dir = data_directory_path_2,
    is_directory_recursive = False,
    batch_size = val_batch_size,
    file_count = val_file_size,
    to_standardize = True,
    include_y_local = False,
    # Labels you want to extract
    labels_list = ['x-midplane', 'y-midplane', 'cotBeta'],  
    # Input shape for the model (time slices, height, width)
    input_shape = (2, 16, 16),
    transpose = (0, 2, 3, 1),
    shuffle = False,
    files_from_end = False,
    
    # New parameters specific to this generator
    tfrecords_dir = tfrecords_dir_validation,
    tfrecords_dir_save = "/home/callea/TFrecords_3sr_qf_slim_2t/val", 
    use_time_stamps = [0, 19],
    #list(range(20)),  # Specific time stamps to use
    quantize = True,  # Whether to use quantization during loading
    max_workers = 2  # For parallel processing
)

Saving batches as TFRecords: 100%|██████████| 14/14 [00:04<00:00,  3.00it/s]


Metadata saved successfully ast /home/callea/TFrecords_3sr_qf_slim_2t/val/metadata.json
Loading metadata from /home/callea/TFrecords_3sr_qf_slim_2t/val/metadata.json


In [8]:
train_generator = OptimizedDataGenerator(
    # Base directory where your parquet files are located
    dataset_base_dir = data_directory_path,
    is_directory_recursive = False,
    batch_size = batch_size,
    file_count = train_file_size,
    to_standardize = True,
    include_y_local = False,
    # Labels you want to extract
    labels_list = ['x-midplane', 'y-midplane', 'cotBeta'],  
    # Input shape for the model (time slices, height, width)
    input_shape = (2, 16,16),
    transpose = (0, 2, 3, 1),
    shuffle = False,
    files_from_end = True,  # Using files from the end of the list
    
    # New parameters specific to this generator
    tfrecords_dir = tfrecords_dir_train,
    tfrecords_dir_save = "/home/callea/TFrecords_3sr_qf_slim_2t/train", 
    use_time_stamps = [0, 19],
    #list(range(20)),  # Specific time stamps to use
    quantize = True,  # Whether to use quantization during loading
    max_workers = 2  # For parallel processing
)

Saving batches as TFRecords: 100%|██████████| 74/74 [00:26<00:00,  2.77it/s]


Metadata saved successfully ast /home/callea/TFrecords_3sr_qf_slim_2t/train/metadata.json
Loading metadata from /home/callea/TFrecords_3sr_qf_slim_2t/train/metadata.json


In [9]:
train_generator.labels_scale

array([99.18727961, 24.80349315,  1.91469723])

In [10]:
input_shape = (16, 16, 2)
model = CreateSlim1DModel(input_shape, n_filters=5)  # Use slim model
model.summary()

2025-07-21 21:15:10.130627: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Model: "smrtpxl_slim"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_pxls/ (InputLayer)    [(None, 16, 16, 2)]          0         []                            
                                                                                                  
 avg_pooling_2d_proj_x (Ave  (None, 16, 1, 2)             0         ['input_pxls/[0][0]']         
 ragePooling2D)                                                                                   
                                                                                                  
 avg_pooling_2d_proj_y (Ave  (None, 1, 16, 2)             0         ['input_pxls/[0][0]']         
 ragePooling2D)                                                                                   
                                                                                       

In [11]:
model.compile(
    optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-3),
    loss=slim_sse_loss,  # Use slim SSE loss
    metrics=[slim_rmse_metric, x_mae_metric, y_mae_metric, cotBeta_mae_metric]  # Optional: add metrics
)

In [12]:
fingerprint = '%08x' % random.randrange(16**8)
os.makedirs("trained_models", exist_ok=True)
base_dir = f'./trained_models/model-{fingerprint}-checkpoints'
os.makedirs(base_dir, exist_ok=True)  
checkpoint_filepath = base_dir + '/weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5'

In [13]:
checkpoint_filepath

'./trained_models/model-d9937291-checkpoints/weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5'

In [14]:
print(fingerprint)

d9937291


In [15]:
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, Callback

early_stopping_patience = 50

class CustomModelCheckpoint(ModelCheckpoint):
    def on_epoch_end(self, epoch, logs=None):
        super().on_epoch_end(epoch, logs)
        checkpoints = [f for f in os.listdir(base_dir) if f.startswith('weights')]
        if len(checkpoints) > 1:
            checkpoints.sort()
            for checkpoint in checkpoints[:-1]:
                os.remove(os.path.join(base_dir, checkpoint))

es = EarlyStopping(patience=early_stopping_patience, restore_best_weights=True)

mcp = CustomModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=True,
    save_freq='epoch',
    verbose=1
)

csv_logger = CSVLogger(f'{base_dir}/training_log.csv', append=True)

In [16]:
class NanMonitorCallback(tf.keras.callbacks.Callback):
    def on_batch_end(self, batch, logs=None):
        logs = logs or {}
        loss = logs.get('loss')
        if loss is not None:
            if np.isnan(loss) or np.isinf(loss):
                print('NaN loss detected, terminating training')
                self.model.stop_training = True
                
# Add this to your callbacks list
nan_monitor = NanMonitorCallback()
callbacks = [mcp, csv_logger, nan_monitor]

In [17]:
# Now use it in model.fit()
history = model.fit(
    x=train_generator,
    validation_data=validation_generator,
    callbacks=callbacks,
    epochs=1000,
    shuffle=False,
    verbose=1
)

Epoch 1/1000


2025-07-21 21:15:14.071560: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2025-07-21 21:15:14.180282: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2025-07-21 21:15:14.527438: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f83f0019d80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-07-21 21:15:14.527494: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-PCIE-40GB MIG 7g.40gb, Compute Capability 8.0
2025-07-21 21:15:14.539441: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1753125314.656190 2797235 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


74/74 [==============================] - ETA: 0s - loss: 2977.1343 - slim_rmse_metric: 0.4365 - x_mae_metric: 0.3935 - y_mae_metric: 0.3545 - cotBeta_mae_metric: 0.3331
Epoch 1: val_loss improved from inf to 1453.85498, saving model to ./trained_models/model-d9937291-checkpoints/weights.01-t2977.13-v1453.85.hdf5
74/74 [==============================] - 10s 94ms/step - loss: 2977.1343 - slim_rmse_metric: 0.4365 - x_mae_metric: 0.3935 - y_mae_metric: 0.3545 - cotBeta_mae_metric: 0.3331 - val_loss: 1453.8550 - val_slim_rmse_metric: 0.3177 - val_x_mae_metric: 0.2565 - val_y_mae_metric: 0.2231 - val_cotBeta_mae_metric: 0.2876
Epoch 2/1000
74/74 [==============================] - ETA: 0s - loss: 928.7441 - slim_rmse_metric: 0.2462 - x_mae_metric: 0.1926 - y_mae_metric: 0.1930 - cotBeta_mae_metric: 0.1922
Epoch 2: val_loss improved from 1453.85498 to 555.07025, saving model to ./trained_models/model-d9937291-checkpoints/weights.02-t928.74-v555.07.hdf5
74/74 [==============================] - 